In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
import psycopg2
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PDFMinerLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_community.chat_message_histories import RedisChatMessageHistory

In [2]:

import logging
import os
import tools_restaurant 

In [3]:
load_dotenv()
DB_CONNECTION = os.getenv("DB_CONNECTION")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
REDIS_URL=os.getenv("REDIS_URL")
chat = ChatOpenAI(model="gpt-4o", temperature=0)
tools=tools_restaurant.TOOLS

In [4]:
SYSTEM_TEMPLATE = """
Eres un chatbot de asistencia al cliente para el restaurante ROOFTOP magdalena

Tendras 2 funciones principales, una para validar los horarios disponibles dada una fecha y una cantidad de personas, y otra para hacer la reserva
de una mesa. 

1. Obtener los horarios disponibles para hacer reservas dada una fecha y una cantidad de personas, y opcionalmente un area(solo si el usuario pregunta)
2. Hacer una reserva. Cuando el usuario quiera hacer una reserva vas a seguir los siguientes pasos :
    a.  Le preguntas la fecha y la cantidad de personas. Tomas la fecha y la conviertes al formato YYYY-MM-DD. Si el usuario no te da el año, asume que es 2024.
    b. Le preguntas si tiene preferencia por alguna zona (main o terraza)
    b. Validas si esa fecha esta disponible y le muestras los horarios.
    c.  Preguntale por sus datos personales, como el nombre, el telefono y el email. 
        Asegurate que el usuario escriba su nombre y apellido
        Valida que el numero telefonico sea correcto, y agrega el sufijo "+57" al telefono del usuario si es necesario,
        Asegurate que la dirección de correo electronico sea valida
    d. Preguntarle si tiene algun comentario con respecto a la reserva para hacercelo saber al restaurante
    e. Vuelve a mostrarle los datos para confirmar la reserva, no la crees hasta que el usuario confirme.No le puedes dar el id de la reserva

La otra de tus funciones sera responder cualquier duda que tenga el cliente 
respecto al menu del restaurante. Para ello podras usar el contexto que está abajo.
<context>
{context}
</context>
"""

In [5]:
def get_session_history(session_id: str) -> RedisChatMessageHistory:
    return RedisChatMessageHistory(session_id, url=REDIS_URL)

In [6]:
def create_chain_agent():

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                SYSTEM_TEMPLATE,
            ),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent= create_openai_tools_agent(chat,tools,prompt)
    agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
    #chain=create_stuff_documents_chain(chat,prompt)
    return agent_executor

In [7]:
def get_chat(chain):
    """ Configura la cadena con historial de mensajes. """
    return RunnableWithMessageHistory(
        chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
    )

In [8]:
def filter_messages(messages,k=10):
    if len(messages)>k:
        return messages[-k:]
    return messages
    

In [9]:
from langchain_core.runnables import RunnablePassthrough

#Create chain
agent_executor = create_chain_agent()



In [10]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'context', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['co

In [11]:
agent_executor_with_message_filter=(RunnablePassthrough.assign(chat_history=lambda x :filter_messages(x["chat_history"]))
 | agent_executor
)

#Get the session history
conversation_chain=get_chat(agent_executor_with_message_filter)

In [53]:
message_body="""
SI"""
wa_id="3227077343"

In [54]:

conversation_chain.invoke({"input": message_body,"context":"",
                            },
                            {"configurable":{"session_id":wa_id}})

Parent run 68360713-4f66-43a8-b8d0-d1ae49e95d36 not found for run 0784afa8-46b2-4482-aa29-93bbce905104. Treating as a root run.




> Entering new AgentExecutor chain...

Invoking: `create_booking` with `{'date': '2024-05-29', 'time': '20:00', 'people': 4, 'phone': '+57 3227077343', 'email': 'pabloaguati@gmail.com', 'name': 'Pablo Guatibonza', 'area_name': 'main'}`


Reserva fallida debido a  b'area cNifDZ9rCHRY8gkiD [bookings.insert: no suitable table in area found]'Parece que no hay una mesa disponible en el área "main" para 4 personas a las 20:00 el 29 de mayo de 2024. 

¿Te gustaría intentar en otra área, como la terraza, o prefieres otro horario?

> Finished chain.


{'input': '\nSI',
 'context': '',
 'chat_history': [AIMessage(content='Perfecto, el 29 de mayo de 2024 está completamente disponible para reservas. Nuestro horario de apertura es de 12:00 PM a 9:00 PM.\n\nPor favor, indícame la hora en la que te gustaría hacer la reserva para 7 personas.'),
  HumanMessage(content='\na la misma hora, mismos datos'),
  AIMessage(content='¡Tu reserva ha sido creada exitosamente, Pablo! Aquí están los detalles de tu nueva reserva:\n\n- **Fecha:** 29 de mayo de 2024\n- **Hora:** 20:00\n- **Personas:** 7\n- **Zona:** Main\n- **Nombre:** Pablo Guatibonza\n- **Teléfono:** +57 3227077343\n- **Correo electrónico:** pabloaguati@gmail.com\n\nSi necesitas hacer algún cambio o tienes alguna otra consulta, no dudes en decírmelo. ¡Gracias por elegir ROOFTOP Magdalena!'),
  HumanMessage(content='\nahora crea otra, pero para 4 personas'),
  AIMessage(content='Vamos a proceder con la creación de la reserva para 4 personas. \n\n¿Te gustaría mantener la misma fecha y hora,